In [2]:
import keras
from keras import layers
from keras import backend as K
from keras.models import Model
import numpy as np

img_shape = (28,28,1)
batch_size = 16
latent_dim = 2  # 잠재 공간의 차원: 2D 평면

input_img = keras.Input(shape=img_shape)

x = layers.Conv2D(32,3,padding='same',activation='relu')(input_img)
x = layers.Conv2D(64,3,padding='same',activation='relu',strides=(2,2))(x)
x = layers.Conv2D(64,3,padding='same',activation='relu')(x)
x = layers.Conv2D(64,3,padding='same',activation='relu')(x)
shape_before_flattening = K.int_shape(x)

x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)

# 입력이미지는 2개의 파라미터로 인코딩
z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

In [5]:
# 잠재 공간 샘플링 함수

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)
    return z_mean + K.exp(0.5*z_log_var)*epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

In [7]:
# 잠재 공간 포인트를 이미지로 매핑하는 VAE 디코더 네트워크

# input에 z 주입
decoder_input = layers.Input(K.int_shape(z)[1:])

# 입력을 업샘플링
x = layers.Dense(np.prod(shape_before_flattening[1:]), activation='relu')(decoder_input)

# 인코더 모델의 마지막 Flatten 층 직전의 특성 맵과 같은 크기를 가진 특성 맵으로 z의 크기를 바꿉니다
x = layers.Reshape(shape_before_flattening[1:])(x)

# Conv2DTranspose층과 Conv2D층을 사용하여 z를 원본 입력 이미지와 같은 크기와 특성 맵으로 디코딩
x = layers.Conv2DTranspose(32, 3, padding='same', activation='relu', strides=(2,2))(x)
x = layers.Conv2D(1, 3, padding='same', activation='sigmoid')(x)

# decoder_input을 디코딩된 이미지로 변환하는 디코더 모델의 객체를 만듭니다
decoder = Model(decoder_input, x)

# 모델에 z를 주입하면 디코딩된 z를 출력
z_decoded = decoder(z)

In [ ]:
# VAE 손실을 계산하기 위해 직접 만든 층
# loss(y_ture, y_pred) 형태는 VAE의 이중 손실에 맞지 않다

class CustomVariationalLayer(keras.layers.Layer):
    
    def vae_loss(self, x, z_decoded):
        